### Air Quality Monitor and Comparison

1. Dataset

Name: Air Quality Data - Updated Hourly
This website provides global air quality data updated hourly on the pollutants of PM2.5, PM10, O3 (Ozone), NO2 (Nitrogen dioxide), SO2 (Sulfur dioxide), CO (Carbon monoxide)

2. Research Questions

How do hourly air pollution patterns (PM₂.₅, NO₂, O₃) vary across major cities, and when do cities experience statistically significant spikes relative to their typical baseline?
We will focus on 6 major cities that allows us to have strong diversity for meaningful comparison across the globe
NYC
London
Hong Kong
Delhi
São Paulo
Tokyo

3. Notebook Link

4. Target Visualization
To be updated
5. Known Unknowns

Known:
Timestamped hourly pollutant readings for (PM2.5, NO2, O3), which allows us to conduct pattern analysis and produce day-of week comparisons, spike detection, and form rolling averages
All pollutants share the same measurement of µg/m³
Geographic coordinates, which allows us to monitor stations for cross-city comparisons, spatial clustering, and mapping in Streamlit
Clearly specified pollutant types allows us to compare traffic-related and climate-related patterns
Real-time data enables a live dashboard for trend and monitoring applications in our application development

Unknown:
Data Quality may vary as OpenAQ aggregates from multiple providers, such as government monitors, low-cost sensors, and private contributors. We are unsure if all monitors are calibrated equally, which may reflect monitoring quality difference when using the data for cross-city comparisons
Station placement bias because the stations are not randomly distributed, as they may be placed near highways, industrial zones, residential areas, etc. that may not represent the entire city fairly. This affects spike interpretation and baseline calculation. We may need to consider pulling data from multiple locations within a city with city-level aggregates, using median values and standard deviations to make the cities more comparable
The definition of spike needs to be constructed
No direct causal variables of traffic counts, weather (temperature and wind), policy changes, industrial output, wildfire data. We may need another set of data(s) to identify the direct causes

6. Anticipated Challenges

Making sure the time stamps are consistent across the data and making sure the time zones are correct
Computing the baseline and spikes correctly, by using city-level hourly aggregation, rolling baseline, and rolling variability estimates. This requires computing the z-score or robust z-score
Need to code to log data pull times and timestamps of a specific time to produce live data on our application


## To be Updated

NYC locations returned: 60
pm25: chosen location_id=384, sensor_id=673, hours returned=200
no2: chosen location_id=853, sensor_id=1523, hours returned=200
o3: chosen location_id=384, sensor_id=671, hours returned=200
raw df shape: (600, 7)
sample hour record (for debugging): {'value': 2.4, 'flagInfo': {'hasFlags': False}, 'parameter': {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'displayName': None}, 'period': {'label': '1hour', 'interval': '01:00:00', 'datetimeFrom': {'utc': '2016-03-12T08:00:00Z', 'local': '2016-03-12T03:00:00-05:00'}, 'datetimeTo': {'utc': '2016-03-12T09:00:00Z', 'local': '2016-03-12T04:00:00-05:00'}}, 'coordinates': None, 'summary': {'min': 2.4, 'q02': 2.4, 'q25': 2.4, 'median': 2.4, 'q75': 2.4, 'q98': 2.4, 'max': 2.4, 'avg': 2.4, 'sd': None}, 'coverage': {'expectedCount': 1, 'expectedInterval': '01:00:00', 'observedCount': 1, 'observedInterval': '01:00:00', 'percentComplete': 100.0, 'percentCoverage': 100.0, 'datetimeFrom': {'utc': '2016-03-12T08:00:00Z', 'local': 

,city,parameter,location_id,sensor_id,datetime,value,unit
200,NYC,no2,853,1523,2016-03-31 18:00:00+00:00,0.009,ppm
201,NYC,no2,853,1523,2016-03-31 19:00:00+00:00,0.011,ppm
202,NYC,no2,853,1523,2016-04-01 02:00:00+00:00,0.010,ppm
203,NYC,no2,853,1523,2016-04-01 03:00:00+00:00,0.009,ppm
204,NYC,no2,853,1523,2016-04-01 04:00:00+00:00,0.004,ppm
205,NYC,no2,853,1523,2016-04-01 05:00:00+00:00,0.003,ppm
206,NYC,no2,853,1523,2016-04-01 06:00:00+00:00,0.003,ppm
207,NYC,no2,853,1523,2016-04-01 07:00:00+00:00,0.004,ppm
208,NYC,no2,853,1523,2016-04-01 08:00:00+00:00,0.004,ppm
209,NYC,no2,853,1523,2016-04-01 09:00:00+00:00,0.007,ppm
